This notebook does the following:

    generate random walk polymer chain
    local optimization using openbabel
    packing of multiple polymer chains into the simulation box
    Compute the system topology and read in the force field data
    write the data file for lammps run

Dependencies:

    pymatgen(latest development version from github)
    Polymers
    packmol

Required input files:

    structures of start, bulk and terminal monomer molecules
    appropriately formatted yaml forcefield data file

In [1]:
import numpy as np

from pprint import pprint

from pymatgen import Molecule
from pymatgen.io.babel import BabelMolAdaptor

from polymers.core import Polymer
from polymers.packmol import PackmolRunner
from polymers.utils import get_topology

Set the start, bulk, and terminal molecules. Also set the forcefield species name mapping

In [2]:
# start molecule
peo_start = Molecule.from_file("PEOmonomer_start.xyz")
s_charges = [-0.1187, 0.0861, 0.0861, 0.0861, -0.2792, -0.0326, 0.0861, 0.0861]
peo_start.add_site_property("charge", s_charges)
s_head = 0
s_tail = 5

# chain molecule
peo_bulk = Molecule.from_file("PEOmonomer_bulk.xyz")
b_charges = [-0.0326, 0.0861, 0.0861, -0.2792, -0.0326, 0.0861, 0.0861]
peo_bulk.add_site_property("charge", b_charges)
head = 0
tail = 4

# terminal molecule
peo_end = Molecule.from_file("PEOmonomer_end.xyz")
e_charges = [-0.0326, 0.0861, 0.0861, -0.2792, -0.1187, 0.0861, 0.0861, 0.0861]
peo_end.add_site_property("charge", e_charges)
e_head = 0
e_tail = 4

# Create polymer via random walk

In [3]:
n_units = 25
link_distance = 1.5075

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance)

# linear chain
peo_polymer_linear = Polymer(peo_start, s_head, s_tail, 
                             peo_bulk, head, tail, 
                             peo_end, e_head, e_tail, 
                             n_units, link_distance, linear_chain=True)

peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")
#peo_polymer_linear.molecule.to(filename="polymer_linear.xyz", fmt="xyz")

set the topology of the polymer from the linear chain

In [4]:
from pymatgen.io.lammps.topology import Topology

topology = Topology.from_molecule(peo_polymer_linear.molecule)

natoms, nbonds, nangles, ndih =  len(topology.atoms), len(topology.bonds), len(topology.angles), len(topology.dihedrals)

print(natoms, nbonds, nangles, ndih)

(177, 176, 325, 366)


# Generate polymer matrix

In [5]:
#Polymer matrix settings

# constituent molecules
molecules = [peo_polymer.molecule]*5
#radius = 100.0
matrix_config = [{"number": 4, "inside box":[0,0,0,150,150,150]},
                 {"number": 3, "inside box":[0,0,0,150,150,150]},
                 {"number": 1, "inside box":[0,0,0,150,150,150]},
                 {"number": 7, "inside box":[0,0,0,150,150,150]},
                 {"number": 2, "inside box":[0,0,0,150,150,150]}]

Use Packmol to pack the polymer chains into a box

In [6]:
pmr = PackmolRunner(molecules,
                    matrix_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    output_file="poly_packed.xyz")
packed_polymer = pmr.run()

packed molecule written to poly_packed.xyz


Read in the force field parameters

In [7]:
from pymatgen.io.lammps.force_field import ForceField

forcefield = ForceField.from_file("ff_data.yaml")
pprint(forcefield.bonds)

{(u'C', u'C'): [1000, 1.5075],
 (u'C', u'H'): [1000, 1.1041],
 (u'C', u'O'): [1000, 1.4115]}


# Generate lammps data file from the topology and the forcefield

In [8]:
from pymatgen.io.lammps.data import LammpsForceFieldData

mols_number = [mol_config["number"] for mol_config in matrix_config]
# sanity check
assert len(packed_polymer) == sum([ len(mol)*mols_number[i] for i, mol in enumerate(molecules)])

box_size = [[0.0, 150], 
            [0.0, 150], 
            [0.0, 150]]

# list of molecular topologies for each constituent molecules
topologies = [topology]*len(molecules)

lammps_ff_data = LammpsForceFieldData.from_forcefield_and_topology(molecules, mols_number, 
                                                                   box_size, packed_polymer, 
                                                                   forcefield, topologies)

# sanity check
for top in topologies:
    assert len(lammps_ff_data.atoms_data) == sum([ len(top.atoms)*mol_number for mol_number in mols_number])
    assert len(lammps_ff_data.bonds_data) == sum([ len(top.bonds)*mol_number for mol_number in mols_number])
    assert len(lammps_ff_data.angles_data) == sum([ len(top.angles)*mol_number for mol_number in mols_number])
    assert len(lammps_ff_data.dihedrals_data) == sum([ len(top.dihedrals)*mol_number for mol_number in mols_number])

Write the data file(uncomment the write line)

In [9]:
print str(lammps_ff_data)

#write the data file
lammps_ff_data.write_data_file("lammps_data.dat")

Data file generated by pymatgen

3009 atoms
2992 bonds
5525 angles
6222 dihedrals

3 atom types
3 bond types
5 angle types
5 dihedral types

0.0 150 xlo xhi
0.0 150 ylo yhi
0.0 150 zlo zhi

Masses 

1 1.00794
2 12.0107
3 15.9994

Pair Coeffs 

1 75844.8 0.2461 396.9
2 33702.4 0.2796 503.0
3 2649.6 0.2674 27.22
4 4320.0 0.2928 137.6
5 14976.0 0.3236 637.6

Bond Coeffs 

1 1000 1.4115
2 1000 1.1041
3 1000 1.5075

Angle Coeffs 

1 42.9 110.1
2 38.5 109.47
3 56.0 109.48
4 86.0 108.54
5 74.5 108.05

Dihedral Coeffs 

1 0.0 0.0 -0.73 0.0
2 0.0 0.0 0.28 0.0
3 1.76 0.67 0.04 0.0
4 0.0 0.0 0.28 0.0
5 0.41 -2.1 -0.6 -0.82

Atoms 

1 1 2 -0.1187 48.5875218069 93.8648132709 77.7210046875
2 1 1 0.0861 47.5396878069 93.6481752709 77.7250186875
3 1 1 0.0861 48.8587608069 94.3666112709 78.6663416875
4 1 1 0.0861 48.8179578069 94.5306992709 76.8709866875
5 1 3 -0.2792 49.3656098069 92.5807182709 77.5859596875
6 1 2 -0.0326 50.7403948069 92.9005322709 77.5839446875
7 1 1 0.0861 51.0116338069 93.40232927